# Import libraries


In [ ]:
from __future__ import absolute_import
import collections
from six.moves import range
import ast
from ast import *
import graphviz
from graphviz import Digraph

In [ ]:
import pandas as pd
import base64
import traceback

# Visualization part


In [ ]:
#Function to obtain the default graph visualization
def default_ast(node, graph, parent=None):

    for child in ast.iter_child_nodes(node):
        child_id = id(child)
        child_name = type(child).__name__
        graph.node(name=str(child_id), label=child_name)
        graph.edge(str(id(node)), str(child_id))
        default_ast(child, graph, node)

In [ ]:
def modified_ast(node, graph, parent=None):
        if isinstance(node, (Load, Store)):
            return

        if isinstance(node, Name):
            if isinstance(parent, (For, comprehension)):
                return

        if isinstance(node, (And, Or)):
            if isinstance(parent, BoolOp):
                return

        if isinstance(node, (Return, Call, Assign, AugAssign, BinOp, For, If, IfExp, Constant, List, BoolOp)):
            if isinstance(parent, (If, For, While, IfExp, comprehension)):
                return

        if isinstance(node, Expr):
            for child in ast.iter_child_nodes(node):
                modified_ast(child, graph, parent)

        else:
            if isinstance(node, Call):
                func_id = id(node.func)
                if node.func.id == "range":
                    return
                graph.node(str(func_id), f"Call: {node.func.id}")
                if parent:
                    graph.edge(str(id(parent)), str(func_id))
                for call_arg in node.args:
                    modified_ast(call_arg, graph, node.func)

            elif isinstance(node, Assign):
                if isinstance(node.targets[0], Tuple) and isinstance(node.value, Tuple):
                    graph.node(str(id(node)-1), f"Tuple")
                    graph.edge(str(id(parent)), str(id(node)-1))

                    for target, value in zip(node.targets[0].elts, node.value.elts):

                        graph.node(str(id(target)-1), "Assign")
                        graph.edge(str(id(node)-1), str(id(target)-1))

                        var_node_id = str(id(target))
                        value_node_id = str(id(value))
                        graph.node(var_node_id, f"Var: {target.id}")
                        if isinstance(value, ast.Constant):
                            graph.node(value_node_id, f"Value: {value.value}")
                        if isinstance(value, ast.Name):
                            graph.node(value_node_id, f"Var: {value.id}")

                        graph.edge(str(id(target)-1), var_node_id)
                        graph.edge(str(id(target)-1), value_node_id)

                else:
                    graph.node(str(id(node)), "Assign")
                    graph.edge(str(id(parent)), str(id(node)))
                    modified_ast(node.targets[0], graph, node)
                    modified_ast(node.value, graph, node)

            elif isinstance(node, AugAssign):
                    graph.node(str(id(node)), "Assign")
                    graph.edge(str(id(parent)), str(id(node)))
                    graph.node(str(id(node.target)+1), f"Var: {node.target.id}")
                    graph.edge(str(id(node)), str(id(node.target)+1))
                    modified_ast(node.op, graph, node)
                    modified_ast(node.target, graph, node.op)
                    modified_ast(node.value, graph, node.op)

            elif isinstance(node, UnaryOp) and isinstance(node.op, USub):
                # Negative constant values
                const_node = node.operand
                if isinstance(const_node, Constant):
                    graph.node(str(id(const_node)), f"Const: -{const_node.value}")
                    if parent:
                        graph.edge(str(id(parent)), str(id(const_node)))
                elif isinstance(const_node, Name):
                    graph.node(str(id(const_node)), f"Var: -{const_node.id}")
                    if parent:
                        graph.edge(str(id(parent)), str(id(const_node)))

            elif isinstance(node, ast.List):
                elements = []
                for elt in node.elts:
                    if isinstance(elt, ast.Constant):
                        elements = ", ".join(str(elt.value) for elt in node.elts)
                    elif isinstance(elt, ast.Name):
                        elements = ", ".join(str(elt.id) for elt in node.elts)

                if len(node.elts)!=0:
                    graph.node(str(id(node)), f"Const: [{elements}]")
                elif len(node.elts)==0:
                    graph.node(str(id(node)), f"Const: []")
                graph.edge(str(id(parent)), str(id(node)))

                for elt in node.elts:
                    if isinstance(elt, ast.BinOp):
                        graph.node(str(id(node)), f"Const: [{elt.left.id} {type(elt.op).__name__} {elt.right.id}]")

            elif isinstance(node, ast.Subscript):
                value_name = node.value.id if isinstance(node.value, ast.Name) else str(node.value)
                graph.node(str(id(node)), f"Var {value_name}")
                graph.edge(str(id(parent)), str(id(node)))
                graph.node(str(id(node.slice)), f"Sliced by")
                graph.edge(str(id(node)), str(id(node.slice)))

                if isinstance(node.slice, ast.Constant):
                    slice_value = node.slice.value if isinstance(node.slice, ast.Constant) else str(node.slice)
                    graph.node(str(id(node.slice.value)), f"Const: {slice_value}")
                    graph.edge(str(id(node.slice)), str(id(node.slice.value)))

                if isinstance(node.slice, ast.BinOp):
                    print(node.slice)
                    modified_ast(node.slice.op, graph, node.slice)
                    modified_ast(node.slice.left, graph, node.slice.op)
                    modified_ast(node.slice.right, graph, node.slice.op)

                if isinstance(node.slice, ast.Name):
                    graph.node(str(id(node.slice.id)), f"Var: {node.slice.id}")
                    graph.edge(str(id(node.slice)), str(id(node.slice.id)))

            else:
                if isinstance(node, For):
                    condition_id = node.iter
                    graph.node(str(id(condition_id)+1), f"Condition:")
                    graph.edge(str(id(node)), str(id(condition_id)+1))

                    graph.node(str(id(node.target)), f"Var: {node.target.id}")
                    graph.edge(str(id(condition_id)+1), str(id(node.target)))

                    if isinstance(node.iter, Call):
                        condition_id = node.iter
                        graph.node(str(id(condition_id)), f"Call: {node.iter.func.id}")
                        graph.edge(str(id(condition_id)+1), str(id(condition_id)))
                        if node.iter.func.id == "range":
                            if isinstance(node.target, ast.Name):
                                for child in node.iter.args:
                                    modified_ast(child, graph, condition_id)

                    if isinstance(node.iter, Name):
                        cond_node = node.iter.id
                        condition_id = node.iter
                        graph.node(str(id(condition_id)), f"Call: In")
                        graph.edge(str(id(condition_id)+1), str(id(condition_id)))

                        graph.node(str(id(cond_node)+1), f"Var: {node.iter.id}")
                        graph.edge(str(id(condition_id)), str(id(cond_node)+1))

                    const_node = node.body
                    if node.body is not None:
                        graph.node(str(id(const_node)), "Body")
                        graph.edge(str(id(node)), str(id(const_node)))
                    for child in node.body:
                        modified_ast(child, graph, const_node)

                for child in ast.iter_child_nodes(node):
                    if not (isinstance(node, Compare) and (
                                                      isinstance(child, Gt) or
                                                      isinstance(child, Lt) or
                                                      isinstance(child, LtE) or
                                                      isinstance(child, GtE) or
                                                      isinstance(child, Eq) or
                                                      isinstance(child, NotEq) or
                                                      isinstance(child, In) or
                                                      isinstance(child, NotIn)
                                                      )) and not (
                      isinstance(node, (BinOp, AugAssign)) and (
                                                      isinstance(child, Mod) or
                                                      isinstance(child, Add) or
                                                      isinstance(child, Sub) or
                                                      isinstance(child, Mult) or
                                                      isinstance(child, Div) or
                                                      isinstance(child, Pow) or
                                                      isinstance(child, FloorDiv))) and not(
                        isinstance(child, arguments) and len(child.args) == 0) and not(
                            isinstance(node, UnaryOp) and isinstance(node.operand, Compare) and isinstance(node.op, Not)
                        ):
                        modified_ast(child, graph, node)

                if isinstance(node, If):
                    condition_id = node.test
                    graph.node(str(id(node)), "If")
                    if isinstance(node.test, BoolOp):
                        graph.node(str(id(node.test)), f"Cond. set: {type(node.test.op).__name__}")
                        graph.edge(str(id(node)), str(id(node.test)))
                        for child in node.test.values:
                            modified_ast(child, graph, node.test)
                            graph.edge(str(id(node.test)), str(id(child)))
                    else:
                        graph.node(str(id(condition_id)+1), f"Condition:")
                        graph.edge(str(id(node)), str(id(condition_id)+1))

                    const_node = node.body
                    if node.body is not None:
                        graph.node(str(id(const_node)), "Body")
                        graph.edge(str(id(node)), str(id(const_node)))
                    for child in node.body:
                        modified_ast(child, graph, const_node)

                    else_node = node.orelse
                    if len(node.orelse)!=0:
                        graph.node(str(id(else_node)), "Else")
                        graph.edge(str(id(node)), str(id(else_node)))
                    for child in node.orelse:
                        modified_ast(child, graph, else_node)


                elif isinstance(node, While):
                    condition_id = node.test
                    graph.node(str(id(node)), "While")
                    graph.node(str(id(condition_id)+1), f"Condition:")
                    graph.edge(str(id(node)), str(id(condition_id)+1))

                    const_node = node.body
                    if node.body is not None:
                        graph.node(str(id(const_node)), "Body")
                        graph.edge(str(id(node)), str(id(const_node)))
                    for child in node.body:
                        modified_ast(child, graph, const_node)

                elif isinstance(node, IfExp):
                    condition_id = node.test
                    graph.node(str(id(condition_id)+1), f"Condition:")
                    graph.edge(str(id(node)), str(id(condition_id)+1))

                    const_node = node.body
                    if node.body is not None:
                        graph.node(str(id(const_node)), "Body")
                        graph.edge(str(id(node)), str(id(const_node)))

                    else_id = id(node.orelse)
                    else_node_id = node.orelse.ctx
                    if node.orelse is not None:
                        graph.node(str(id(else_node_id)), "Else")
                        graph.edge(str(id(node)), str(id(else_node_id)))
                        modified_ast(node.orelse, graph, else_node_id)

                elif isinstance(node, comprehension):
                    condition_id = node.iter
                    graph.node(str(id(condition_id)), f"Condition:")
                    graph.edge(str(id(node)), str(id(condition_id)))

                    graph.node(str(id(node.target)), f"Var: {node.target.id}")
                    graph.edge(str(id(condition_id)), str(id(node.target)))

                    if isinstance(node.iter, Call):
                        cond_node = node.iter.func.id
                        graph.node(str(id(cond_node)), f"Call: {cond_node}")
                        graph.edge(str(id(condition_id)), str(id(cond_node)))
                        if node.iter.func.id == "range":
                            if isinstance(node.target, ast.Name):
                                for child in node.iter.args:
                                    modified_ast(child, graph, cond_node)

                    if isinstance(node.iter, Name):
                        cond_node = node.iter.id
                        graph.node(str(id(cond_node)), f"Call: In")
                        graph.edge(str(id(condition_id)), str(id(cond_node)))

                        graph.node(str(id(cond_node)+1), f"Var: {node.iter.id}")
                        graph.edge(str(id(cond_node)), str(id(cond_node)+1))

                    const_node = node.ifs
                    if node.ifs is not None:
                        graph.node(str(id(const_node)+1), "Body")
                        graph.edge(str(id(node)), str(id(const_node)+1))
                        for child in node.ifs:
                            graph.node(str(id(child)+1), "If")
                            graph.edge(str(id(const_node)+1), str(id(child)+1))
                            if isinstance(child, ast.Compare):
                                graph.node(str(id(child)+2), f"Condition:")
                                graph.edge(str(id(child)+1), str(id(child)+2))
                                graph.edge(str(id(child)+2), str(id(child)))
                            elif isinstance(child, ast.BoolOp):
                                graph.node(str(id(child)+2), f"Cond. set: {type(child.op).__name__}")
                                graph.edge(str(id(child)+1), str(id(child)+2))
                                for value in child.values:
                                    modified_ast(value, graph, str(id(child)+2))
                                    graph.edge(str(id(child)+2), str(id(value)))

                if isinstance(node, Compare) and isinstance(node.ops[0], Gt):
                    graph.node(str(id(node)), "Compare: >")
                elif isinstance(node, Compare) and isinstance(node.ops[0], Lt):
                    graph.node(str(id(node)), "Compare: <")
                elif isinstance(node, Compare) and isinstance(node.ops[0], LtE):
                    graph.node(str(id(node)), "Compare: <=")
                elif isinstance(node, Compare) and isinstance(node.ops[0], GtE):
                    graph.node(str(id(node)), "Compare: >=")
                elif isinstance(node, Compare) and isinstance(node.ops[0], Eq):
                    graph.node(str(id(node)), "Compare: ==")
                elif isinstance(node, Compare) and isinstance(node.ops[0], NotEq):
                    graph.node(str(id(node)), "Compare: !=")
                elif isinstance(node, Compare) and isinstance(node.ops[0], In):
                    graph.node(str(id(node)), "Call: In")
                elif isinstance(node, Compare) and isinstance(node.ops[0], NotIn):
                    graph.node(str(id(node)), "Condition: Not In")
                elif isinstance(node, UnaryOp) and isinstance(node.operand, Compare) and isinstance(node.op, Not):
                    graph.node(str(id(node)), "Call: Not In")
                    for child in ast.iter_child_nodes(node.operand):
                      if not isinstance(child, ast.In):
                        modified_ast(child, graph, node)
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, Mod)) or isinstance(node, Mod):
                    graph.node(str(id(node)), f"Operation: %")
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, Add)) or isinstance(node, Add):
                    graph.node(str(id(node)), f"Operation: +")
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, Sub)) or isinstance(node, Sub):
                    graph.node(str(id(node)), f"Operation: -")
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, Mult)) or isinstance(node, Mult):
                    graph.node(str(id(node)), f"Operation: *")
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, Div)) or isinstance(node, Div):
                    graph.node(str(id(node)), f"Operation: /")
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, FloorDiv)) or isinstance(node, FloorDiv):
                    graph.node(str(id(node)), f"Operation: //")
                elif (isinstance(node, (BinOp, AugAssign)) and isinstance(node.op, Pow)) or isinstance(node, Pow):
                    graph.node(str(id(node)), f"Operation: **")

                elif isinstance(node, Name):
                    if parent and isinstance(parent, (For, comprehension)):
                        for child in ast.iter_child_nodes(parent):
                            if isinstance(child, Call):
                                for call_arg in child.args:
                                    if call_arg.value == node.id:
                                        graph.edge(str(id(child)), str(id(node)))
                                        return
                    else:
                        graph.node(str(id(node)), f"Var: {node.id}")

                else:
                    graph.node(str(id(node)), str(node.__class__.__name__))

                if isinstance(node, (Compare, UnaryOp)) and isinstance(parent, (If, IfExp, While)):
                    condition_id = parent.test
                    graph.edge(str(id(condition_id)+1), str(id(node)))

                elif isinstance(node, (If, IfExp, While)) and isinstance(node.test, Call):
                    condition_id = node.test
                    cond_node = node.test.func.id
                    graph.node(str(id(cond_node)), f"Call: {cond_node}")
                    graph.edge(str(id(condition_id)+1), str(id(cond_node)))

                if parent:
                  if not isinstance(node, (Compare, Call,)) and  not isinstance(parent, (If, IfExp, While)):
                    graph.edge(str(id(parent)), str(id(node)))

                if isinstance(node, Constant):
                    value = node.value
                    if type(value) is str:
                        formatted_value = f"'{value}'"
                    else:
                        formatted_value = str(value)
                    graph.node(str(id(node)), f"Const: {formatted_value}")
                elif isinstance(node, Name):
                    graph.node(str(id(node)), f"Var: {node.id}")
                elif isinstance(node, FunctionDef ):
                    graph.node(str(id(node)), f"Function: {node.name}")
                elif isinstance(node, arg):
                    graph.node(str(id(node)), f"Arg: {node.arg}")
                elif isinstance(node, alias):
                    graph.node(str(id(node)), f"{node.name}")
                elif isinstance(node, Return):
                    graph.node(str(id(node)), f"Call: return")
                    if isinstance(node.value, Compare):
                      graph.edge(str(id(node)), str(id(node.value)))

# FUNCTION FOR CUSTOMMIZATION NODE CLASS


In [ ]:
def get_children(node):
    if isinstance(node, list):
        return node
    return node.children

def get_label(node):
    return str(node.label)

In [ ]:
def handle_comparison(comparison_node):
    comparison_ops = {
        ast.Gt: '>',
        ast.Lt: '<',
        ast.LtE: '<=',
        ast.GtE: '>=',
        ast.Eq: '==',
        ast.NotEq: '!=',
        ast.In: 'In',
        ast.NotIn: 'Not in'
    }
    operator = comparison_node.ops[0]
    operator_label = comparison_ops.get(type(operator), '')

    left_operand = ast_to_tree(comparison_node.left)
    right_operand = ast_to_tree(comparison_node.comparators[0])

    condition_children = [left_operand, right_operand]
    condition_child=[]
    if operator_label not in('In', 'Not in'):
        condition_child.append(Node(f"Compare: {operator_label}", condition_children))
    if operator_label in('In', 'Not in'):
        condition_child.append(Node(f"Call: {operator_label}", condition_children))
    return condition_child


def ast_to_tree(ast_node):
    node_type = type(ast_node).__name__
    if node_type == 'Load' or node_type == 'Store':
        return None

    elif node_type == 'Expr':
        # For 'Expr' nodes, directly add their children to the parent node
        children = []
        for child in ast.iter_child_nodes(ast_node):
            child_node = ast_to_tree(child)
            if child_node is not None:
                children.append(child_node)
        return children

    elif node_type == 'UnaryOp':
        # For 'ast.UnaryOp' nodes, handle the 'ast.USub' separately
        if isinstance(ast_node.op, ast.USub):
            operand_node = ast_to_tree(ast_node.operand)
            if isinstance(ast_node.operand, ast.Constant):
                return Node(f"Const: -{ast_node.operand.value}")
            elif isinstance(ast_node.operand, ast.Name):
                return Node(f"Var: -{ast_node.operand.id}")
        else:
            return None

    elif isinstance(ast_node, ast.FunctionDef):
        children = []
        for child in ast.iter_child_nodes(ast_node):
            child_node = ast_to_tree(child)
            if child_node is not None:
                if isinstance(child_node, list):
                    children.extend(child_node)
                else:
                    children.append(child_node)
        return Node(f"Function: {ast_node.name}", children=children)

    elif isinstance(ast_node, ast.arg):
        return Node(f"Arg: {ast_node.arg}")

    elif isinstance(ast_node, ast.arguments):
        children = []
        for child in ast.iter_child_nodes(ast_node):
              child_node = ast_to_tree(child)
              if child_node is not None:
                  if isinstance(child_node, list):
                      children.extend(child_node)
                  else:
                      children.append(child_node)
        if len(children)!=0:
          return Node(f"arguments", children=children)
        else:
          return None

    elif isinstance(ast_node, (ast.BinOp, ast.AugAssign)):
        children = []

        if isinstance(ast_node.op, ast.Mod):
            node_label = "Operation: %"
        elif isinstance(ast_node.op, ast.Add):
            node_label = "Operation: +"
        elif isinstance(ast_node.op, ast.Sub):
            node_label = "Operation: -"
        elif isinstance(ast_node.op, ast.Mult):
            node_label = "Operation: *"
        elif isinstance(ast_node.op, ast.Div):
            node_label = "Operation: /"
        elif isinstance(ast_node.op, ast.FloorDiv):
            node_label = "Operation: //"
        elif isinstance(ast_node.op, ast.Pow):
            node_label = "Operation: **"

        if isinstance(ast_node, ast.BinOp):
            # Process the left and right operands recursively
            left_operand_children = ast_to_tree(ast_node.left)
            right_operand_children = ast_to_tree(ast_node.right)

            # Append the left operand children to the current node's children
            if left_operand_children is not None:
                children.append(left_operand_children)

            # Append the right operand children to the current node's children
            if right_operand_children is not None:
                children.append(right_operand_children)

            return Node(node_label, children=children)

        elif isinstance(ast_node, ast.AugAssign):
            target_children = ast_to_tree(ast_node.target)
            value_children = ast_to_tree(ast_node.value)

            if target_children is not None:
                children.append(target_children)
            if value_children is not None:
                children.append(value_children)

            return Node("Assign", children=[Node("Var: " + ast_node.target.id),
                                             Node("Var: " + ast_node.target.id),
                                             Node("Const:" + str(ast_node.value.value)),
                                            Node(node_label)])


    elif isinstance(ast_node, ast.For):
        children = []

        # Handling the loop variable and iteration range/iterable
        loop_variable = ast_to_tree(ast_node.target)
        loop_range = None
        if isinstance(ast_node.iter, ast.Call) and isinstance(ast_node.iter.func, ast.Name) and ast_node.iter.func.id == "range":
            range_args = [ast_to_tree(arg) for arg in ast_node.iter.args]
            loop_range = Node("Call: range", children=range_args)
        if isinstance(ast_node.iter, ast.Name):
            cond_node = ast_node.iter.id
            loop_range = Node("Call: In", children=[Node("Var: " + cond_node)])

        # Loop condition node
        condition_node = Node("Condition:", children=[loop_variable, loop_range])
        children.append(condition_node)

        # Handling the body of the for loop
        body_children = []
        for statement in ast_node.body:
            body_node = ast_to_tree(statement)
            if body_node:
                if isinstance(body_node, list):
                    body_children.extend(body_node)
                else:
                    body_children.append(body_node)
        if body_children:
            children.append(Node("Body:", children=body_children))

        # Create the For node
        return Node("For", children=children)


    elif isinstance(ast_node, ast.Assign):
        children = []

        if isinstance(ast_node.targets[0], ast.Tuple) and isinstance(ast_node.value, ast.Tuple):
            node_label = "Tuple"
            for target, value in zip(ast_node.targets[0].elts, ast_node.value.elts):
                assign_children = []

                assign_node_label = "Assign"
                target_node_label = f"Var: {target.id}"
                if isinstance(value, ast.Constant):
                  value_node_label = f"Const: {value.value}"
                if isinstance(value, ast.Name):
                  value_node_label = f"Var: {value.id}"

                assign_children.append(Node(target_node_label))
                assign_children.append(Node(value_node_label))

                children.append(Node(assign_node_label, children=assign_children))

        else:
            node_label = "Assign"
            target_node_label = f"Var: {ast_node.targets[0].id}"
            value_node = ast_to_tree(ast_node.value)

            children.append(Node(target_node_label))
            if value_node is not None:
                children.append(value_node)

        return [Node(node_label, children=children)]

    elif isinstance(ast_node, (ast.If, ast.While)):
        children = []

        # Handle the test condition
        if isinstance(ast_node.test, ast.Compare):
            children.append(Node(f"Condition:", children=handle_comparison(ast_node.test)))

        elif isinstance(ast_node.test, ast.BoolOp):
            # Handling for BoolOp (logical AND, OR)
            boolop_children = []
            for value in ast_node.test.values:
                if isinstance(value, ast.Compare):
                    boolop_children.extend(handle_comparison(value))
                else:
                    condition_child = ast_to_tree(value)
                    if condition_child:
                        if isinstance(condition_child, list):
                            boolop_children.extend(condition_child)
                        else:
                            boolop_children.append(condition_child)
            if boolop_children:
                children.append(Node(f"Cond. set: {type(ast_node.test.op).__name__}", children=boolop_children))

        elif isinstance(ast_node.test, ast.UnaryOp) and isinstance(ast_node.test.operand, ast.Compare) and isinstance(ast_node.test.op, ast.Not):
            # Handling for UnaryOp with Not
            children.append(Node("Condition:"))

            # Process the left and right operands recursively
            left_operand_children = ast_to_tree(ast_node.test.operand.left)
            right_operand_children = ast_to_tree(ast_node.test.operand.comparators[0])

            # Append the left operand children to the current node's children
            if left_operand_children:
                if isinstance(left_operand_children, list):
                    children.extend(left_operand_children)
                else:
                    children.append(left_operand_children)

            # Append the right operand children to the current node's children
            if right_operand_children:
                if isinstance(right_operand_children, list):
                    children.extend(right_operand_children)
                else:
                    children.append(right_operand_children)
            children.append(Node("Call: Not In"))
            children.append(Node("Condition:"))
        else:
            test_node = ast_to_tree(ast_node.test)
            if test_node:
                children.append(Node("Condition:", children=[test_node]))

        # Handle the body
        body_children = []
        for statement in ast_node.body:
            body_node = ast_to_tree(statement)
            if body_node:
                if isinstance(body_node, list):
                    body_children.extend(body_node)
                else:
                    body_children.append(body_node)
        if body_children:
            children.append(Node("Body:", children=body_children))

        # Handle the else part if there is
        if hasattr(ast_node, 'orelse'):
            else_children = []
            for statement in ast_node.orelse:
                else_node = ast_to_tree(statement)
                if else_node:
                    if isinstance(else_node, list):
                        else_children.extend(else_node)
                    else:
                        else_children.append(else_node)
            if else_children:
                children.append(Node("Else:", children=else_children))

        # Create and return the node
        return Node(node_type, children=children)


    elif isinstance(ast_node, ast.List):
        elements = []
        for elt in ast_node.elts:
            if isinstance(elt, ast.Constant):
                elements.append(str(elt.value))
            elif isinstance(elt, ast.Name):
                elements.append(elt.id)

        elements_str = ', '.join(elements)

        if elements:
            return Node(f"Const: [{elements_str}]")
        else:
            return Node("Const: []")

    elif isinstance(ast_node, ast.Subscript):
        children = []
        child = []
        if isinstance(ast_node.slice, (ast.Constant, ast.Name, ast.BinOp)):
            child.append(ast_to_tree(ast_node.slice))
        children.append(Node("Sliced by", child))

        return Node(f"Var: {ast_node.value.id}", children=children)

    elif isinstance(ast_node, ast.Compare):
        # Handling ast.Compare nodes
        return handle_comparison(ast_node)

    else:
        if isinstance(ast_node, ast.Constant):
            # For 'ast.Constant' nodes, include the formatted value in the label
            value = ast_node.value
            if isinstance(value, str):
                formatted_value = f"'{value}'"
            else:
                formatted_value = str(value)
            label = f"Const: {formatted_value}"

        elif isinstance(ast_node, ast.Call):
            # New condition for handling 'Call' nodes
            children = []
            node_label = node_type

            # Iterating through each argument
            for arg in ast_node.args:
                child = ast_to_tree(arg)  # Note: call ast_to_tree, not ast_to_zss_node
                if child:
                    if isinstance(child, list):
                        children.extend(child)
                    else:
                        children.append(child)  # Add the children at the same level

            # Checking if the function has an id and updating the node_label
            if hasattr(ast_node.func, "id"):
                node_label = f"Call: {ast_node.func.id}"

            # Return a node representing the function call with its arguments as children
            return Node(node_label, children=children)

        elif isinstance(ast_node, ast.Name):
            label = f"Var: {ast_node.id}"

        elif isinstance(ast_node, ast.Module):
            label = f"Module"

        elif isinstance(ast_node, ast.Return):
            label = f"Call: return"

        elif isinstance(ast_node, ast.alias):
            return Node(f"alias: {ast_node.name}")

        # Special handling for ast.ImportFrom nodes
        elif isinstance(ast_node, ast.ImportFrom):
            names = [ast_to_tree(alias) for alias in ast_node.names]
            return Node(f"ImportFrom: {ast_node.module}", children=names)

        else:
            label = str(type(ast_node))
        node = Node(label)
        for child in ast.iter_child_nodes(ast_node):
            child_node = ast_to_tree(child)
            if child_node is not None:
                if isinstance(child_node, list):
                    for cn in child_node:
                        node.addkid(cn)
                else:
                    node.addkid(child_node)

        return node

# Cost function

In [ ]:
labels = ['General mismatch', 'Mismatch in print arguments', 'Mismatch in constant values', 'Constant value is in the wrong place', 'Mismatch in variable names', 'Function call mismatch',
          'General value mismatch', 'Mismatch in function arguments', 'Missing code block', 'Mismatch in if condition', 'Mismatch in loop condition']
costs = [1, 10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000, 10000000000]

# Combine labels and costs into a list of tuples
label_cost_tuples = list(zip(labels, costs))

print(label_cost_tuples)

[('General mismatch', 1), ('Mismatch in print arguments', 10), ('Mismatch in constant values', 100), ('Constant value is in the wrong place', 1000), ('Mismatch in variable names', 10000), ('Function call mismatch', 100000), ('General value mismatch', 1000000), ('Mismatch in function arguments', 10000000), ('Missing code block', 100000000), ('Mismatch in if condition', 1000000000), ('Mismatch in loop condition', 10000000000)]


In [ ]:
def get_cost_by_label(label):
    for l, cost in label_cost_tuples:
        if l == label:
            return cost
    return 1  # Default cost if label not found in the list

def get_label_by_cost(cost):
    if cost == 0:
        return 'Match operation'
    for label, c in label_cost_tuples:
        if c == cost:
            return label

def strdist(a, b, parent_a, parent_b, tree_A, tree_B):
    # Extract the parts before the colon
    prefix_a = a.split(':', 1)[0]
    prefix_b = b.split(':', 1)[0]

    # Check if the prefixes are the same
    if prefix_a == prefix_b:
        if prefix_a == "Const":
            if parent_a==parent_b=="Call: print":
                if a != b:
                  return get_cost_by_label("Mismatch in print arguments")
                else:
                  return 0

            elif parent_a==parent_b!="Call: print":
                if a != b:
                  return get_cost_by_label("Mismatch in constant values")
                else:
                  return 0

            elif parent_a!=parent_b:
                if a != b:
                  return get_cost_by_label("Constant value is in the wrong place")
                else:
                  return 0

        elif prefix_a == "Var":
            if a != b:
                return get_cost_by_label("Mismatch in variable names")
            else:
                return 0

        elif prefix_a == "Call":
            if a != b:
                return get_cost_by_label("Function call mismatch")
            else:
                return 0

        elif prefix_a == "Module":  # Handle Module nodes separately
            return 0

        elif a==b:
            return 0

        else:
            return get_cost_by_label("General value mismatch")

    elif prefix_a == "Arg" or prefix_b == "Arg":
        if a != b:
            return get_cost_by_label("Mismatch in function arguments")
        else:
            return 0

    elif prefix_a in ["Body", "Condition", "Assign"] or prefix_b in ["Body", "Condition", "Assign"]:
        return 0  # Set cost to 0 for these cases

    elif (prefix_a == "" or prefix_b == "") and not (prefix_a in ["Body", "Condition", "Assign", "For", "While", "If"] or prefix_b in ["Body", "Condition", "Assign", "For", "While", "If"]):
        return get_cost_by_label("Missing code block")

    elif prefix_a == "If" or prefix_b == "If":
        if a != b:
            return get_cost_by_label("Mismatch in if condition")
        else:
            return 0

    elif prefix_a == "For" or prefix_b == "For" or prefix_a == "While" or prefix_b == "While":
        if a != b:
            return get_cost_by_label("Mismatch in loop condition")
        else:
            return 0

    else:
        return get_cost_by_label("General mismatch")  # Default cost for other cases

# ZSS NODE CLASS


In [ ]:
class Node(object):

    def __init__(self, label, children=None, parent=None):
        self.label = label
        self.children = []
        self.parent = parent
        if children is not None:
            for child in children:
                self.addkid(child)

    def addkid(self, node):
        self.children.append(node)

    def __str__(self, level=0):
        s = ""
        if level > 0:
            s += " " * (4 * level - 2)
        s += "%d:%s" % (len(self.children), self.label)
        for child in self.children:
            s += "\n" + child.__str__(level + 1)
        return s

    @staticmethod
    def get_children(node):
        if node is None:
            return []
        if isinstance(node, list):
            return node
        if hasattr(node, 'children'):
            return node.children
        if callable(node):
            return node()
        return []

    @staticmethod
    def get_parent(node):
        """
        Get the label of the parent node of the given node.
        """
        if node is not None and node.parent is not None:
            return node.parent.label
        else:
            return None

    @staticmethod
    def get_label(node):
        """
        Default value of ``get_label`` argument of :py:func:`zss.distance`.

        :returns: ``self.label``.
        """
        if node is None:
            return None
        if isinstance(node, list):
            return node
        return node.label

    def addkid(self, node, before=False):
        """
        Add the given node as a child of this node.
        """
        if before:  self.children.insert(0, node)
        else:   self.children.append(node)
        node.parent = self  # Set the parent attribute of the child node
        return self

    def get(self, label):
        """:returns: Child with the given label."""
        if self.label == label: return self
        for c in self.children:
            if label in c: return c.get(label)

    def iter(self):
        """Iterate over this node and its children in a preorder traversal."""
        queue = collections.deque()
        queue.append(self)
        while len(queue) > 0:
            n = queue.popleft()
            for c in n.children: queue.append(c)
            yield n

    def __contains__(self, b):
        if isinstance(b, str) and self.label == b: return 1
        elif not isinstance(b, str) and self.label == b.label: return 1
        elif (isinstance(b, str) and self.label != b) or self.label != b.label:
            return sum(b in c for c in self.children)
        raise TypeError("Object %s is not of type str or Node" % repr(b))

    def __eq__(self, b):
        if b is None:
            return False
        if not isinstance(b, Node):
            return False
        return self.label == b.label

    def __ne__(self, b):
        return not self.__eq__(b)

    def __repr__(self):
        return super(Node, self).__repr__()[:-1] + " %s>" % self.label

    def __str__(self):
        s = "%d:%s" % (len(self.children), self.label)
        s = '\n'.join([s]+[str(c) for c in self.children])
        return s

    def __sub__(self, other):
        return tree_distance.simple_distance(self, other)

In [ ]:
try:
    import numpy as np
    zeros = np.zeros
except ImportError:
    def py_zeros(dim, pytype):
        assert len(dim) == 2
        return [[pytype() for y in range(dim[1])]
                for x in range(dim[0])]
    zeros = py_zeros

class AnnotatedTree(object):

    def __init__(self, root, get_children, get_parent):
        self.get_children = get_children
        self.get_parent = get_parent
        self.root = root
        self.nodes = list()  # a post-order enumeration of the nodes in the tree
        self.ids = list()    # a matching list of ids
        self.lmds = list()   # left most descendents
        self.keyroots = None
            # k and k' are nodes specified in the post-order enumeration.
            # keyroots = {k | there exists no k'>k such that lmd(k) == lmd(k')}
            # see paper for more on keyroots

        stack = list()
        pstack = list()
        stack.append((root, collections.deque()))
        j = 0
        while len(stack) > 0:
            n, anc = stack.pop()
            nid = j
            for c in self.get_children(n):
                a = collections.deque(anc)
                a.appendleft(nid)
                stack.append((c, a))
            pstack.append(((n, nid), anc))
            j += 1
        lmds = dict()
        keyroots = dict()
        i = 0
        while len(pstack) > 0:
            (n, nid), anc = pstack.pop()
            self.nodes.append(n)
            self.ids.append(nid)
            if not self.get_children(n):
                lmd = i
                for a in anc:
                    if a not in lmds: lmds[a] = i
                    else: break
            else:
                try: lmd = lmds[nid]
                except:
                    import pdb
                    pdb.set_trace()
            self.lmds.append(lmd)
            keyroots[lmd] = i
            i += 1
        self.keyroots = sorted(keyroots.values())

    def print_postorder_numbering(self):
        for i, node in enumerate(self.nodes):
            if isinstance(node, list):
                for subnode in node:
                    print(f'Node Label: {subnode.label}, Postorder Number: {i}')
            else:
                print(f'Node Label: {node.label}, Postorder Number: {i}')

class Operation(object):
    """
    Dummy class for storing edit operations
    """
    remove = 0
    insert = 1
    update = 2
    match = 3

    def __init__(self, op, arg1=None, arg2=None):
        self.type = op
        self.arg1 = arg1
        self.arg2 = arg2

    def __repr__(self):
        arg1_label = getattr(self.arg1, 'label', None)
        arg2_label = getattr(self.arg2, 'label', None)

        if self.type == self.remove:
            return f'<Operation Remove: {arg1_label}>' if arg1_label is not None else '<Operation Remove>'
        elif self.type == self.insert:
            return f'<Operation Insert: {arg2_label}>' if arg2_label is not None else '<Operation Insert>'
        elif self.type == self.update:
            return f'<Operation Update: {arg1_label} to {arg2_label}>' if arg1_label is not None and arg2_label is not None else '<Operation Update>'
        else:
            return f'<Operation Match: {arg1_label} to {arg2_label}>' if arg1_label is not None and arg2_label is not None else '<Operation Match>'


    def __eq__(self, other):
        if other is None or not isinstance(other, Operation):
            return False
        return self.type == other.type and self.arg1 == other.arg1 and self.arg2 == other.arg2

REMOVE = Operation.remove
INSERT = Operation.insert
UPDATE = Operation.update
MATCH = Operation.match

# TreeDistance

In [ ]:
class TreeDistance:
    def __init__(self):
        self.operations = None
        self.treedists = None
        self.costs_list = None
        self.error_labels = None

    def distance(self, A, B, get_children, get_parent, insert_cost, remove_cost, update_cost,
                 return_operations=False):

        A, B = AnnotatedTree(A, get_children, get_parent), AnnotatedTree(B, get_children, get_parent)
        size_a = len(A.nodes)
        size_b = len(B.nodes)
        self.treedists = zeros((size_a, size_b), float)
        self.operations = [[[] for _ in range(size_b)] for _ in range(size_a)]
        self.costs_list = [[[] for _ in range(size_b)] for _ in range(size_a)]
        self.error_labels = [[[] for _ in range(size_b)] for _ in range(size_a)]

        def treedist(i, j):
          Al = A.lmds
          Bl = B.lmds
          An = A.nodes
          Bn = B.nodes

          m = i - Al[i] + 2
          n = j - Bl[j] + 2
          fd = zeros((m,n), float)
          partial_ops = [[[] for _ in range(n)] for _ in range(m)]
          partial_costs = [[[] for _ in range(n)] for _ in range(m)]
          partial_error_labels = [[[] for _ in range(n)] for _ in range(m)]

          ioff = Al[i] - 1
          joff = Bl[j] - 1

          for x in range(1, m): # δ(l(i1)..i, θ) = δ(l(1i)..1-1, θ) + γ(v → λ)
              node = An[x+ioff]
              fd[x][0] = fd[x-1][0] + remove_cost(node)
              op = Operation(REMOVE, node)
              partial_ops[x][0] = partial_ops[x-1][0] + [op]
              partial_error_labels[x][0] = partial_error_labels[x-1][0] + [get_label_by_cost(remove_cost(node))]
          for y in range(1, n): # δ(θ, l(j1)..j) = δ(θ, l(j1)..j-1) + γ(λ → w)
              node = Bn[y+joff]
              fd[0][y] = fd[0][y-1] + insert_cost(node)
              op = Operation(INSERT, arg2=node)
              partial_ops[0][y] = partial_ops[0][y-1] + [op]
              partial_error_labels[0][y] = partial_error_labels[0][y-1] + [get_label_by_cost(insert_cost(node))]

          for x in range(1, m):  # the plus one is for the xrange impl
              for y in range(1, n):
                  # x+ioff in the fd table corresponds to the same node as x in
                  # the treedists table (same for y and y+joff)
                  node1 = An[x+ioff]
                  node2 = Bn[y+joff]
                  # only need to check if x is an ancestor of i
                  # and y is an ancestor of j
                  if Al[i] == Al[x+ioff] and Bl[j] == Bl[y+joff]:
                      #                   +-
                      #                   | δ(l(i1)..i-1, l(j1)..j) + γ(v → λ)
                      # δ(F1 , F2 ) = min-+ δ(l(i1)..i , l(j1)..j-1) + γ(λ → w)
                      #                   | δ(l(i1)..i-1, l(j1)..j-1) + γ(v → w)
                      #                   +-
                      costs = [fd[x-1][y] + remove_cost(node1),
                              fd[x][y-1] + insert_cost(node2),
                              fd[x-1][y-1] + update_cost(node1, node2)]
                      fd[x][y] = min(costs)
                      min_index = costs.index(fd[x][y])

                      if min_index == 0:
                          op = Operation(REMOVE, node1)
                          partial_ops[x][y] = partial_ops[x-1][y] + [op]
                          partial_costs[x][y] = partial_costs[x-1][y] + [costs[min_index]]
                          partial_error_labels[x][y] = partial_error_labels[x-1][y] + [get_label_by_cost(remove_cost(node1))]
                      elif min_index == 1:
                          op = Operation(INSERT, arg2=node2)
                          partial_ops[x][y] = partial_ops[x][y - 1] + [op]
                          partial_costs[x][y] = partial_costs[x][y - 1] + [costs[min_index]]
                          partial_error_labels[x][y] = partial_error_labels[x][y - 1] + [get_label_by_cost(insert_cost(node2))]
                      else:
                          op_type = MATCH if fd[x][y] == fd[x-1][y-1] else UPDATE
                          op = Operation(op_type, node1, node2)
                          partial_ops[x][y] = partial_ops[x - 1][y - 1] + [op]
                          partial_costs[x][y] = partial_costs[x-1][y-1] + [costs[min_index]]
                          partial_error_labels[x][y] = partial_error_labels[x - 1][y - 1] + [get_label_by_cost(update_cost(node1, node2))]
                          #else:
                           # partial_error_labels[x][y] = partial_error_labels[x - 1][y - 1] + ["Match operation"]

                      self.operations[x + ioff][y + joff] = partial_ops[x][y]
                      self.treedists[x+ioff][y+joff] = fd[x][y]
                      self.costs_list[x+ioff][y+joff] = partial_costs[x][y]
                      self.error_labels[x+ioff][y+joff] = partial_error_labels[x][y]

                  else:
                      #                   +-
                      #                   | δ(l(i1)..i-1, l(j1)..j) + γ(v → λ)
                      # δ(F1 , F2 ) = min-+ δ(l(i1)..i , l(j1)..j-1) + γ(λ → w)
                      #                   | δ(l(i1)..l(i)-1, l(j1)..l(j)-1)
                      #                   |                     + treedist(i1,j1)
                      #                   +-
                      p = Al[x+ioff]-1-ioff
                      q = Bl[y+joff]-1-joff
                      costs = [fd[x-1][y] + remove_cost(node1),
                              fd[x][y-1] + insert_cost(node2),
                              fd[p][q] + self.treedists[x+ioff][y+joff]]
                      fd[x][y] = min(costs)
                      min_index = costs.index(fd[x][y])
                      if min_index == 0:
                          op = Operation(REMOVE, node1)
                          partial_ops[x][y] = partial_ops[x-1][y] + [op]
                          partial_costs[x][y] = partial_costs[x-1][y] + [costs[min_index]]
                          partial_error_labels[x][y] = partial_error_labels[x-1][y] + [get_label_by_cost(remove_cost(node1))]
                      elif min_index == 1:
                          op = Operation(INSERT, arg2=node2)
                          partial_ops[x][y] = partial_ops[x][y-1] + [op]
                          partial_costs[x][y] = partial_costs[x][y-1] + [costs[min_index]]
                          partial_error_labels[x][y] = partial_error_labels[x][y - 1] + [get_label_by_cost(insert_cost(node2))]
                      else:
                          partial_ops[x][y] = partial_ops[p][q] + self.operations[x+ioff][y+joff]
                          partial_costs[x][y] = partial_costs[x-1][y-1] + [costs[min_index]]
                          partial_error_labels[x][y] = partial_error_labels[p][q] + self.error_labels[x+ioff][y+joff]
                          #elif op_type != 2:
                            #partial_error_labels[x][y] = partial_error_labels[x - 1][y - 1] + ["Match operation"]

        for i in A.keyroots:
            for j in B.keyroots:
                treedist(i, j)

        if return_operations:
            return self.treedists[-1][-1], self.operations[-1][-1], self.error_labels[-1][-1]
        else:
            return self.treedists[-1][-1]

    def simple_distance(self, A, B, get_children=Node.get_children, get_parent=Node.get_parent,
                    get_label=Node.get_label, label_dist=strdist, return_operations=False):
          self.distance(
              A, B, get_children, get_parent,
              insert_cost=lambda node: label_dist('', get_label(node), None, get_parent(node), A, B),
              remove_cost=lambda node: label_dist(get_label(node), '', get_parent(node), None, A, B),
              update_cost=lambda a, b: label_dist(get_label(a), get_label(b), get_parent(a), get_parent(b), A, B),
              return_operations=return_operations
          )
          if return_operations:
              return self.treedists[-1][-1], self.operations[-1][-1], self.error_labels[-1][-1]
          else:
              return self.treedists[-1][-1]

tree_distance = TreeDistance()
tree_distance.operations = None
tree_distance.treedists = None
tree_distance.costs_list = None
tree_distance.error_labels = None

# Test


In [ ]:
code1 = """
def panier(p):
  if p >= 50 :
    return p
  else :
    return p+4.99
"""

code2 = """
def blabla(n):
    if n<5:
        for k in range(n):
            print ("bla")
"""

In [ ]:
from collections import OrderedDict

#First we need to parse our python code using ast.parse() to create an AST. However the built-in AST modules generates ASTs that aren't directly compatible with the zss library
#Therefore, we need to convert the AST into a tree of 'Node' objects that zss library can handle.
ast1 = ast.parse(code1)
ast2 = ast.parse(code2)

#Then we need to traverse these AST to create a corresponding 'Node' tree.
tree1 = ast_to_tree(ast1)
tree2 = ast_to_tree(ast2)
distance, operations, error_labels = tree_distance.simple_distance(tree2, tree1, return_operations=True)
print(distance)
print(error_labels)
print(operations)
df_graph = graphviz.Digraph()
df_graph2 = graphviz.Digraph()
modified_ast(ast1, df_graph, None)
modified_ast(ast2, df_graph2, None)
df_graph.render("ast_tree1", format="png")
df_graph2.render("ast_tree2", format="png")

# Count the occurrences of each error label and store the costs in an OrderedDict
error_counts = OrderedDict()
error_costs = OrderedDict()
for error_label in error_labels:
    if error_label == "Match operation":
        continue  # Skip the rest of the loop for this iteration

    if error_label not in error_counts:
        error_counts[error_label] = 0
        print(0)
    error_counts[error_label] += 1
    print(error_counts[error_label])
    for label, cost in label_cost_tuples:
        if error_label == label:
            error_costs[error_label] = cost
            break
    else:
        # If no match is found, set the cost to 0
        error_costs[error_label] = 0

# Convert the OrderedDicts to separate lists for labels, counts, and costs
labels_list = list(error_counts.keys())
counts_list = list(error_counts.values())
costs_list = list(error_costs.values())

distance_value = 0
for count, cost in zip(counts_list, costs_list):
    if count <= 9:
      distance_value += count*cost
    if count > 9:
      distance_value += 9*cost

if distance_value == 0:
    labels_list = ["Correct code"]
    counts_list = [1]
    costs_list = [0]

print("Distance value:")
print(distance_value)
print("Error labels:")
print(labels_list)
print("Error counts:")
print(counts_list)
print("Error costs:")
print(costs_list)

803111001.0
['General value mismatch', 'Match operation', 'Mismatch in variable names', 'Constant value is in the wrong place', 'General value mismatch', 'Match operation', 'Match operation', 'Missing code block', 'Missing code block', 'Missing code block', 'Match operation', 'General mismatch', 'Function call mismatch', 'Match operation', 'Match operation', 'Missing code block', 'Missing code block', 'Missing code block', 'Missing code block', 'Missing code block', 'Match operation', 'Match operation', 'General value mismatch', 'Match operation']
[<Operation Update: Arg: n to Arg: p>, <Operation Match: arguments to arguments>, <Operation Update: Var: n to Var: p>, <Operation Update: Const: 5 to Const: 50>, <Operation Update: Compare: < to Compare: >=>, <Operation Insert: Condition:>, <Operation Remove: Condition:>, <Operation Remove: Var: k>, <Operation Remove: Var: n>, <Operation Remove: Call: range>, <Operation Remove: Condition:>, <Operation Update: Const: 'bla' to Var: p>, <Operat

# File extraction


In [ ]:
import pandas as pd
import ast
from collections import OrderedDict
import traceback

# Read the datasets into DataFrames
df_test = pd.read_csv('resultant_file.csv', encoding='ISO-8859-1')
#df_correct = pd.read_excel('correct_codes.xlsx')

# Merge datasets based on 'id_exercice'
#merged_df = df_test.merge(df_correct, on='id_exercice', how='inner')

# List to store indices of rows to be deleted
rows_to_delete = []

# Process each row in the merged DataFrame
for index, row in df_test.iterrows():
    code1 = row['code']
    code2 = row['correct_code']

    try:

        print(row['id'])
        # Parse the code and create the AST
        ast1 = ast.parse(code1)
        ast2 = ast.parse(code2)

        # Convert AST to tree
        tree1 = ast_to_tree(ast1)
        tree2 = ast_to_tree(ast2)

        distance, operations, error_labels = tree_distance.simple_distance(tree1, tree2, return_operations=True)

        # Count the occurrences of each error label and store the costs in an OrderedDict
        error_counts = OrderedDict()
        error_costs = OrderedDict()
        for error_label in error_labels:
            if error_label == "Match operation":
                continue  # Skip the rest of the loop for this iteration

            if error_label not in error_counts:
                error_counts[error_label] = 0
            error_counts[error_label] += 1
            for label, cost in label_cost_tuples:
                if error_label == label:
                    error_costs[error_label] = cost
                    break
            else:
                # If no match is found, set the cost to 0
                error_costs[error_label] = 0

        # Convert the OrderedDicts to separate lists for labels, counts, and costs
        labels_list = list(error_counts.keys())
        counts_list = list(error_counts.values())
        costs_list = list(error_costs.values())

        distance_value = 0
        for count, cost in zip(counts_list, costs_list):
            if count <= 9:
              distance_value += count*cost
            if count > 9:
              distance_value += 9*cost

        if distance_value == 0:
            labels_list = ["Correct code"]
            counts_list = [1]
            costs_list = [0]

        # Concatenate operations and costs into a string
        errors_str = '\n'.join(f"{error}" for error in labels_list)

        # Concatenate operations and costs into a string
        counts_str = '\n'.join(f"{count}" for count in counts_list)

        # Concatenate operations and costs into a string
        costs_str = '\n'.join(f"{cost}" for cost in costs_list)

        # Add the results to the corresponding rows in the DataFrame
        df_test.loc[index, 'New_distance'] = distance_value
        df_test.loc[index, 'New_error_list'] = errors_str
        df_test.loc[index, 'New_counts_list'] = counts_str
        df_test.loc[index, 'New_costs_list'] = costs_str

    except (SyntaxError, AttributeError, TypeError):
        print(row['id_exercice'])
        print(row['code'])
        traceback.print_exc()

        # Add the index of the problematic row to the list
        rows_to_delete.append(index)

# Drop the problematic rows from the merged DataFrame
df_test.drop(rows_to_delete, inplace=True)

# Write the updated merged DataFrame back to the file
df_test.to_csv('report.csv', index=False)

250
259
261
263
267
268
468
469
472
586
587
588
590
591
593
594
599
600
868
1343
1344
1347
1639
1642
1643
1644
2240
2241
2526
2621
2622
2623
2626
2627
3088
3089
3093
3095
3097
3098
3099
3100
3101
3627
4239
4246
4247
4249
4251
4254
4295
4298
4300
4302
4306
4307
4308
4309
4492
4617
4734
4735
4736
4737
4738
4739
4885
4887
4888
4889
4890
4891
4892
4893
4896
4898
4899
4900
4903
5139
5169
5177
5190
8394
8401
8411
8422
8432
8449
8458
8896
8914
8917
8923
9110
9125
9132
10550
10551
10554
10555
10556
10557
10558
10632
10880
10883
10891
10898
11191
11195
11197
11206
11513
11547
11566
11571
12433
12450
12451
12455
12472
12483
12487
12494
12502
12506
14521
14527
14528
14529
14531
14533
14546
14551
15285
15350
15792
15801
15805
16008
16167
16171
16172
16176
16178
16180
16575
16578
16579
16580
16581
24543
24732
24774
24775
24781
24791
24804
24937
24942
25101
25150
25177
25197
25206
25217
25227
25303
25438
26793
26798
26811
26824
26828
26830
27498
27509
27996
28009
28016
28019
28020
28022
28028
28047


Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 2
    droite(2)    
    ^^^^^^
IndentationError: expected an indented block after 'while' statement on line 1
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    else:
         ^
IndentationError: unindent does not match any outer indentation level


2726
2728
2731
2833
152
while nonArrive() :
    if murDroite() :
        haut (1)
        else :
            droite (1)
2834
3174
3605
3611
11881
11957
11977
12965
152
while nonArrive() :
     if murDroite() :
         haut(1)
    avancer(1)     
12969
12972
12974
152
while nonArrive() :
     if murDroite() :
         haut(1)
    droite(1)     
12976
152
while nonArrive() :
     if murDroite() :
         haut(1)
    else :
        droite(1)
12977
13754


Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    else :
    ^^^^
SyntaxError: invalid syntax
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    avancer(1)     
                   ^
IndentationError: unindent does not match any outer indentation level
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    droite(1)     
                  ^
Indenta

16731
22886
22906
22915
22957
22965
23007
23020
23026
23076
23091
23339
23356
23361
23364
23376
23387
23404
24349
26609
152
while nonArrive() :
    if murDroite() :
        haut(1)
    else droite(1)
26614
27110
27118
27168


...

35247
35250
35267
35274
35438
152
while nonArrive() :
      if murDroite() :
          
          
         

35670
152
while nonArrive() :
      if murDroite() :
          haut(1)
          else :
              droite(1)
         
35681
152
while nonArrive() :
      if murDroite() :
          haut(1)
        else :
              droite(1)
         
35687
152
while nonArrive() :
      if murDroite() :
          haut(1)
    else :
              droite(1)
         
35699
35769
35813
35891


Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 5
    
    ^
IndentationError: expected an indented block after 'if' statement on line 2
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    else :
    ^^^^
SyntaxError: invalid syntax
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    else :
          ^
IndentationError: unindent does not match any o

36922
37489
37692
37695
37697
37699
37702
37703
37982
37983
37984
37985
37986
37988
37989
37991
38278
38280
38289
38299
152
while nonArrive() :
if murDroite(1):
        droite(1)
    else:
        haut(1)
38320
38695
38696
38845
38918
39673
39676


Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 2
    if murDroite(1):
    ^^
IndentationError: expected an indented block after 'while' statement on line 1
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    else : 
    ^^^^
SyntaxError: invalid syntax
Traceback (most recent call last):
  File "<ipython-input-14-816da1f653ca>", line 25, in <cell line: 17>
    ast1 = ast.parse(code1)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 4
    else : 
    ^^^^
SyntaxError: invalid syntax


39685
152
while nonArrive():
    if murDroite() :
        haut (1)
        else : 
    droite (1)

39690
152
while nonArrive():
    if murDroite() :
        haut (1)
        else : 
            droite (1)

39694
41052
9121
9136
9160
9171
9189
9198
9206
9225
9245
9258
9263
9266
9267
9268
9269
9272
9276
9277
9279
9282
9283
9284
9285
9287
9288
9289
9292
9293
9295
9297
9299
9304
9306
9307
9310
9319
9320
9321
9323
9324
9326
9327
9328
9329
9330
9332
9333
9334
9336
9337
9338
9343
9346
9347
9348
9350
9353
9355
9359
9362
9364
9366
9373
9376
9378
9379
9381
9382
9383
9384
9385
9386
9388
9389
9390
9391
9392
9394
9395
9397
9398
9399
9400
9401
9402
9403
9404
9405
9406
9407
9408
9409
9411
9412
9413
9414
9415
9416
9417
9418
9419
9420
9421
9422
9424
9425
9426
9427
9428
9429
9430
9431
9432
9433
9434
9436
29168
29169
38972
38973
38985
38992
38993
38996
38998
39002
39004
1169
1279
1280
1282
1283
1416
2375
2377
2479
2534
2536
2850
3065
3066
3612
3729
3732
4108
4109
4110
4112
4113
4119
4303
4304
4310
6127
6

# Bubble chart

In [ ]:
pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pyvis.network import Network
import json
import matplotlib.cm as cm
import matplotlib

In [ ]:
def get_color_from_count(count, max_count):
    colormap = cm.get_cmap('Blues')  # or any other colormap you prefer from matplotlib
    scaled_value = count / max_count
    color_rgba = colormap(scaled_value)
    return matplotlib.colors.rgb2hex(color_rgba)

# Read the dataset
df = pd.read_csv('report.csv')

# Convert date_created to datetime
df['date_created'] = pd.to_datetime(df['date_created'], format='%d/%m/%Y %H:%M')

# Take user input for the exercise ID
#exercise_id = int(input("Enter the exercise ID: "))

# Filter the dataset based on the exercise ID
#df = df[df['id_exercice'] == exercise_id]

# Continue only if there are records for the given exercise_id
if df.empty:
    print(f"No records found for exercise ID: {exercise_id}")
else:
    # Sorting based on student id and date
    df = df.sort_values(['id_compte', 'date_created'])

# Collect edges and nodes
edges = {}
nodes = {}

# Iterate over students
for student, group in df.groupby('id_compte'):
    previous_error = None
    for idx, row in group.iterrows():
        error = row['New_error_list']

        # Update nodes count
        nodes[error] = nodes.get(error, 0) + 1

        if previous_error:
            edge_key = (previous_error, error)
            edges[edge_key] = edges.get(edge_key, 0) + 1

        previous_error = error

# Create the graph
nt = Network(notebook=True, height="750px", width="100%", bgcolor="#fff3de", font_color="black")

# Physics configuration to increase the distance between nodes
options = {
    "physics": {
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "springLength": 200,  # Adjust this value to increase/decrease distance
            "gravitationalConstant": -50,
            "damping": 0.4,
            "avoidOverlap": 0.5
        },
        "minVelocity": 0.75,
        "timestep": 0.9
    }
}

nt.set_options(json.dumps(options))

max_node_count = max(nodes.values())

# Add nodes and edges to the graph
for node, count in nodes.items():
    color = get_color_from_count(count, max_node_count)
    nt.add_node(node, label=f"{node}\n({count})", value=count, color=color)

for edge, count in edges.items():
    nt.add_edge(edge[0], edge[1], title=str(count), value=count, label=str(count), arrows='to')

# Display
nt.save_graph("network2.html")

<ipython-input-17-862ae9297824>:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('Blues')  # or any other colormap you prefer from matplotlib


# New Section

In [ ]:
import ast
code = """
def panier(p):
    if p >= 50 :
        return p
    else :
        return p+4.99
"""
tree = ast.parse(code)
print(ast.dump(tree, indent=4))

Module(
    body=[
        FunctionDef(
            name='panier',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='p')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                If(
                    test=Compare(
                        left=Name(id='p', ctx=Load()),
                        ops=[
                            GtE()],
                        comparators=[
                            Constant(value=50)]),
                    body=[
                        Return(
                            value=Name(id='p', ctx=Load()))],
                    orelse=[
                        Return(
                            value=BinOp(
                                left=Name(id='p', ctx=Load()),
                                op=Add(),
                                right=Constant(value=4.99)))])],
            decorator_list=[])],
    type_ignor